# Pêndulo de Newton

*Modelagem e Simulação do Mundo Físico*

Grupo: Antônio Amaral, Arthur Barreto, Ana Laiz Farias, Luan Cintra

*#Importando Bibliotecas*

In [36]:
#Numpy
import numpy as np
#MatPlotLib
import matplotlib.pyplot as plt
#Math
from math import *
#Solve_IVP
from scipy.integrate import solve_ivp

*#Medidas Padrões*

In [37]:
L = 0.06
g = 10
m = 0.0064
D = 0.04
K = 0.92

*#Modelo*

In [38]:
def modelo(t,lista):
    x = lista[0]
    y = lista[1]
    vx = lista[2]
    vy = lista[3]
    v = sqrt( vx**2 + vy**2 )
    cosseno = ( L - y ) / L
    seno = - x / L
    T = m * g * cosseno + m * v**2 / L
    dxdt = vx
    dydt = vy
    dvxdt = T / m * seno
    dvydt = - g + T / m * cosseno
    
    return [ dxdt , dydt , dvxdt , dvydt ]
    
def cond_parada(t, lista):
    return lista[0]

cond_parada.terminal = True

def encontra_t_max(lista_t,lista_y):
    ymax = lista_y[0]
    tmax = 0
    for i in range(len(lista_t)):
        if lista_y[i]>ymax:
            ymax = lista_y[i]
            tmax = lista_t[i]
    return tmax

*#Calculos*

In [39]:
L = 0.06 #Comprimento de L
x_0 = -7.344e-2 #X Inicial
y_0 = 2.725e-2 #Y Inicial

sen = y_0/L #Calculando Seno
cos = sqrt(1 - sen**2) #Calculando Cosseno

*#Testando Execução Bilateral*

In [40]:
tf = 2 #Tempo Final
dt = 1e-3 #Delta Tempo
t_span = ( 0 , tf )
lista_t = np.arange( 0, tf, dt)

CI = [ -L*cos, L*sen+0.005 , 0 , 0 ] #Condições Iniciais

solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

lista_x = solucao.y[ 0 , : ] #X
lista_y = solucao.y[ 1 , : ] #Y
lista_vx = solucao.y[ 2 , : ] #VX

plt.plot( lista_x , lista_y , 'r')

CI = [ 1e-4 , 0 , K * lista_vx[-1] , 0 ]

solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

lista_x = solucao.y[ 0 , : ] + D  # D FOI SOMADO PARA REPRESENTAR A POSIÇÃO DA BOLINHA DA DIREITA
lista_y = solucao.y[ 1 , : ]

plt.plot( lista_x , lista_y, 'b')
plt.grid()
plt.axis("equal")
plt.show()

### Implementação do Modelo com Perda de energia

*#Introduzindo Listas*

In [41]:
lista_ymax = []
lista_t_ymax = []

*#Variaveis*

In [42]:
tfinal = 0

*#Condições Iniciais*

In [43]:
CI = [ -L*cos , L*sen+0.005 , 0 , 0 ]

*#Loop Principal*

In [44]:
for i in range(50):

    if i%2 == 0 :

        solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

        lista_x = solucao.y[ 0 , : ]
        lista_y = solucao.y[ 1 , : ]
        lista_vx = solucao.y[ 2 , : ]

        lista_ymax.append(max(lista_y))
        lista_t_ymax.append(encontra_t_max( solucao.t , lista_y ) + tfinal)

        tfinal = tfinal + solucao.t[-1]
        
        plt.plot( lista_x , lista_y ) 

        CI = [ 1e-4 , 0 , K * lista_vx[-1] , 0 ]
        
    else:
    
        solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

        lista_x = solucao.y[ 0 , : ] + D
        lista_y = solucao.y[ 1 , : ]
        lista_vx = solucao.y[ 2 , : ]

        lista_ymax.append(max(lista_y))
        lista_t_ymax.append(encontra_t_max( solucao.t , lista_y ) + tfinal)

        tfinal = tfinal + solucao.t[-1]
        
        plt.plot( lista_x , lista_y ) 
        
        CI = [ -1e-4 , 0 , K * lista_vx[-1] , 0 ]

#Plot Gráfico
#Movimento do Pendulo
plt.axis("equal")
plt.title("Movimento do pendulo")
plt.grid(True)
plt.show() 

#Altura Máxima em Função do Tempo
plt.plot( lista_t_ymax , lista_ymax, 'ro' )
plt.title("Altura máxima em função do tempo")
plt.grid(True)
plt.show() 



### Validação do Modelo

*#Importações*

In [45]:
import pandas

*#Recebendo e Definindo Medidas Experimentais*

In [46]:
colunas = ["t","x","y"] #,"vx","vy"]
datae = pandas.read_csv("Dados_MassA.csv",names = colunas)
datad = pandas.read_csv("Dados_MassB.csv",names = colunas)

te = datae.t.tolist()
# ajustar a origem no eixo do tempo

t_e = []
for t in te:
    t_e.append(t - 8.133)
    
xe = datae.x.tolist()
ye = datae.y.tolist()
# VAMOS CALCULAR A ALTURA MÁXIMA DE CADA COLISÃO
y_e_max = [ye[0],ye[33],ye[66],ye[98],ye[128],ye[161],ye[193],ye[224],ye[258],ye[289]]  
t_e_ok = [t_e[0],t_e[33],t_e[66],t_e[98],t_e[128],t_e[161],t_e[193],t_e[224],t_e[258],t_e[289]]

# ajustar a origem no eixo do tempo para o lado direito
td = datad.t.tolist()

t_d = []
for t in td:
    t_d.append(t - 8.275)
        
xd = datad.x.tolist()
yd = datad.y.tolist()
y_d_max = [yd[18],yd[50],yd[82],yd[115],yd[147],yd[179],yd[210],yd[240],yd[273],yd[305]]
t_d_ok = [t_d[18],t_d[50],t_d[82],t_d[115],t_d[147],t_d[179],t_d[210],t_d[240],t_d[273],t_d[305]]

# print(ye[0])
# print(lista_ymax[0])
d = lista_ymax[0] - ye[0]
#print("Devemos deslocar {} no eixo".format(d))
y = []
for x in lista_ymax:
    y.append(x - d)


*#Plotando Gráficos*

In [47]:
plt.title("Resultado da primeira iteração")
plt.plot(t_e_ok,y_e_max,"mo", label = "Bola Esquerda")
plt.plot(t_d_ok,y_d_max,"bo", label = "Bola Direita")
plt.plot(lista_t_ymax , y, 'r', label = "Teórico")
plt.legend()
plt.xlabel("Tempo (s)")
plt.ylabel("Altura máxima (m)")
plt.grid(True)
plt.show()

#### Verificando Qual a Melhor Constante de Perda (K)

*#Criando Lista de K*

In [48]:
lisK = np.arange(0.96,0.965,0.001)

*#Loop Principal*

In [49]:
for K in lisK:
    
    CI = [ -L*cos , L*sen + 0.005, 0 , 0 ]
    
    lista_ymax_1 = []
    lista_t_ymax_1 = []
    tfinal_1 = 0

    for i in range(50):

        if i%2 == 0 :

            solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

            lista_x = solucao.y[ 0 , : ]
            lista_y = solucao.y[ 1 , : ]
            lista_vx = solucao.y[ 2 , : ]

            lista_ymax_1.append(max(lista_y))
            lista_t_ymax_1.append(encontra_t_max( solucao.t , lista_y ) + tfinal_1)

            tfinal_1 = tfinal_1 + solucao.t[-1]

            #plt.plot( lista_x , lista_y ) 

            CI = [ 1e-4 , 0 , K * lista_vx[-1] , 0 ]

        else:

            solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

            lista_x = solucao.y[ 0 , : ] + D
            lista_y = solucao.y[ 1 , : ]
            lista_vx = solucao.y[ 2 , : ]

            lista_ymax_1.append(max(lista_y))
            lista_t_ymax_1.append(encontra_t_max( solucao.t , lista_y ) + tfinal_1)

            tfinal_1 = tfinal_1 + solucao.t[-1]

            #plt.plot( lista_x , lista_y ) 

            CI = [ -1e-4 , 0 , K * lista_vx[-1] , 0 ]
            
    # agora plotar os graficos corretos
    
    y = []
    for x in lista_ymax_1:
        y.append(x - d)
    
    plt.plot( lista_t_ymax_1 , y, label = "K = {}".format(K))

#Plot Gráfico - Variação de K
plt.plot(t_e_ok,y_e_max,"mo", label = "Bola Esquerda")
plt.plot(t_d_ok,y_d_max,"bo", label = "Bola Direita") 
plt.title("Variando o K")
plt.xlabel("Tempo (s)")
plt.ylabel("Altura máxima (m)")
plt.grid(True)
plt.legend()
plt.show()

#### Criando Modelo com K Calibrado

*#Listas*

In [50]:
lista_ymax_2 = []
lista_t_ymax_2 = []
y1 = []

*#Condições Iniciais*

In [51]:
CI = [ -L*cos , L*sen + 0.005, 0 , 0 ]

*#Variaveis*

In [52]:
tfinal_2 = 0
K = 0.963

*#Loop Principal*

In [53]:
for i in range(50):

    if i%2 == 0 :

        solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

        lista_x = solucao.y[ 0 , : ]
        lista_y = solucao.y[ 1 , : ]
        lista_vx = solucao.y[ 2 , : ]

        lista_ymax_2.append(max(lista_y))
        lista_t_ymax_2.append(encontra_t_max( solucao.t , lista_y ) + tfinal_2)

        tfinal_2 = tfinal_2 + solucao.t[-1]

        #plt.plot( lista_x , lista_y ) 

        CI = [ 1e-4 , 0 , K * lista_vx[-1] , 0 ]

    else:

        solucao = solve_ivp( modelo , t_span , CI , events = cond_parada , t_eval = lista_t )

        lista_x = solucao.y[ 0 , : ] + D
        lista_y = solucao.y[ 1 , : ]
        lista_vx = solucao.y[ 2 , : ]

        lista_ymax_2.append(max(lista_y))
        lista_t_ymax_2.append(encontra_t_max( solucao.t , lista_y ) + tfinal_2)

        tfinal_2 = tfinal_2 + solucao.t[-1]

        #plt.plot( lista_x , lista_y ) 

        CI = [ -1e-4 , 0 , K * lista_vx[-1] , 0 ]

# agora plotar os graficos corretos


for x in lista_ymax_2:
    y1.append(x - d)

plt.plot( lista_t_ymax_2 , y1, "r", label = "K = {}".format(K))
plt.plot(t_e_ok,y_e_max,"mo", label = "Bola Esquerda")
plt.plot(t_d_ok,y_d_max,"bo", label = "Bola Direita") 
plt.title("Validação do Modelo")
plt.xlabel("Tempo (s)")
plt.ylabel("Altura máxima (m)")
plt.grid(True)
plt.legend()
plt.show() 

### Gráfico Conclusivo do Modelo

*#Importações*

In [54]:
#Ferramentas 3D
from mpl_toolkits.mplot3d import Axes3D
#MatPlotLib
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
%matplotlib qt5

*#Listas*

In [55]:
#Lista de Alpha (Massas)
lista_alpha = np.linspace(6.4e-3,8,10)
#Lista de Beta (K)
lista_beta = np.linspace(0,1,10)
#Lista de Máximo
ListaMaximo = []

print(len(lista_beta),len(lista_alpha))

10 10


## Modelo

*Neste Modelo acrescentamos a opção de variar a massa das bolas do Pêndulo*

In [56]:
def modelo1(t,lista,M):
    x = lista[0]
    y = lista[1]
    vx = lista[2]
    vy = lista[3]
    v = sqrt( vx**2 + vy**2 )
    cosseno = ( L - y ) / L
    seno = - x / L
    T = M * g * cosseno + M * v**2 / L
    dxdt = vx
    dydt = vy
    dvxdt = T / M * seno
    dvydt = - g + T / M * cosseno
    Dxdt = [dxdt,dydt,dvxdt,dvydt]
    return Dxdt
def cond_parada(t, lista,M):
    return lista[0]

cond_parada.terminal = True

*#Loop Principal*

In [59]:
fig = plt.figure()
ax = fig.gca(projection = "3d")

for K in lista_beta:
    for M in lista_alpha:
        CI = [ -L*cos , L*sen + 0.005, 0 , 0 ]
        lista_ymax_3 = []
        lista_t_ymax_3 = []
        tfinal_3 = 0
        for i in range(50):

            if i%2 == 0:

                solucao = solve_ivp( modelo1 , t_span , CI , events = cond_parada , t_eval = lista_t, args = (M,) )

                lista_x = solucao.y[ 0 , : ]
                lista_y = solucao.y[ 1 , : ]
                lista_vx = solucao.y[ 2 , : ]

                lista_ymax_3.append(max(lista_y))
                lista_t_ymax_3.append(encontra_t_max( solucao.t , lista_y ) + tfinal_3)

                tfinal_3 = tfinal_2 + solucao.t[-1]

                CI = [ 1e-4 , 0 , K * lista_vx[-1] , 0 ]

            else:

                solucao = solve_ivp( modelo1 , t_span , CI , events = cond_parada , t_eval = lista_t, args = (M,) )

                lista_x = solucao.y[ 0 , : ] + D
                lista_y = solucao.y[ 1 , : ]
                lista_vx = solucao.y[ 2 , : ]

                lista_ymax_3.append(max(lista_y))
                lista_t_ymax_3.append(encontra_t_max( solucao.t , lista_y ) + tfinal_3)

                tfinal_3 = tfinal_3 + solucao.t[-1]

                CI = [ -1e-4 , 0 , K * lista_vx[-1] , 0 ]


        y1 = []
        for x in lista_ymax_2:
            y1.append(x - d)
        
X,Y = np.meshgrid(lista_alpha,lista_beta)
Z = np.array (y1)

#Plot - Surperficie
surf = ax.plot_surface(X,Y,Z,cmap = cm.coolwarm, linewidth = 0, antialised=True)

#Customizando eixo Z
ax.set_zlim(5, 5)
ax.zaxis.set_major_locator(LinearLocator(10))
ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))



# Add a color bar which maps values to colors.
fig.colorbar(surf, shrink=0.5, aspect=5)

ax.set_xlabel('d1')
ax.set_ylabel('d2')
ax.set_zlabel('Máximo de T2')

plt.show()


ValueError: Argument Z must be 2-dimensional.